## 곱셈 & 덧셈 계층 구현


In [ ]:
# Multiple

class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
  def forward(self, x, y):
    self.x = x
    self.y = y
    return x * y
  def backward(self, out):
    dx = out * self.y
    dy = out * self.x

    return dx, dy

In [ ]:
class AddLayer:
  def __init__(self):
    pass
  def forward(self, x, y):
    return x + y
  def backward(self, out):
    dx = out * 1
    dy = out * 1
    return dx, dy

## 활성화함수

In [ ]:
# ReLU

class Relu:
    def __init__(self):
      self.mask = None
    def forward(self, x):
      self.mask = (x <= 0)
      out = x.copy()
      out[self.mask] = 0
      return out
    def backward(self, out):
      out[self.mask] = 0
      x = out
      return x